# Calculate GHG concentrations

IPCC AR6 methodology:

The following description comes from the Excel sheet of long-lived greenhouse gas concentrations, v9. See https://github.com/chrisroadmap/ar6/blob/main/data_input/observations/LLGHG_history_AR6_v9_for_archive.xlsx

All values are mid-year mean.

**While we wait for the updates from Xin Lan and Jens Muhle, we do 2023 estimates using an extrapolation. This is for the purposes of the forcing time series only.**

NOAA only:
- CO2
- CH3CCl3  (Chris override AR6)
- CCl4 (Chris override AR6)
- CFC-11 (Chris override AR6)
- CFC-12 (Chris override AR6)

AGAGE only:
- CFC-114
- CFC-115
- CFC-13
- CF4
- C2F6
- C3F8
- c-C4F8
- NF3
- SO2F2
- HFC-23
- HFC-236ea (?)
- HFC-245fa
- HFC-43-10mee
- CHCl3
- c-C4F8    xxx  (Note that there is ~7% calibration difference for c-C4F8 between AGAGE (Muhle et al. (2019) and Droste et al (2020), but it cannot be resolved using a simple scaling factor, therefore only AGAGE results are included here.)

Merged NOAA and AGAGE:
- CH4          
- N2O
- HFC-134a
- HFC-32
- HFC-125
- HFC-143a
- HFC-152a
- HFC-227ea  
- HFC-365mfc
- SF6
- CFC-113
- HCFC-22
- HCFC-141b
- HCFC-142b
- CH2Cl2        xxx
- CH3Cl         xxx !!!
- CH3Br
- Halon-1211
- Halon-1301
- Halon-2402
- HCFC-133a     xxx !!! (AGAGE HCFC-133a were adjusted down 7% for account for ~14% calibration difference between AGAGE and Laube et al (2014), in an attempt to express HCFC-133a as average of AGAGE and UEA estimates.)


xxx : data is not available from the aggregated 
!!! : data is not available from the disaggreated


Laube et al. (2014) and WMO (2018) Southern Hemisphere, extrapolation needed to PD
- CFC-112
- CFC-112a
- CFC-113a

Laube et al. (2014) and WMO (2018), extrapolation needed to PD
- CFC-114a

Schoenberger et al. (2015), extrapolation needed to PD
- HCFC-31

Simmonds et al. (2017), extrapolation needed to PD
- HCFC-124

Droste et al. (2020). CMIP6 scaled to Droste et al. (2020) to account for calibration change.
- n-C4F10
- n-C5F12
- n-C6F14
- i-C6F14
- C7F16	

Vollmer et al. (2018), extrapolation needed to PD
- C8F18

https://gml.noaa.gov/aftp/data/ is usually a good place to look

NOAA (accessed 2024-02-29):
- https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_gl.txt
- https://gml.noaa.gov/webdata/ccgg/trends/ch4/ch4_annmean_gl.txt
- https://gml.noaa.gov/webdata/ccgg/trends/n2o/n2o_annmean_gl.txt
- https://gml.noaa.gov/webdata/ccgg/trends/sf6/sf6_annmean_gl.txt
- https://gml.noaa.gov/aftp/data/hats/Total_Cl_Br/2023%20update%20total%20Cl%20Br%20&%20F.xls  (converted to CSV)

AGAGE (accessed 2023-03-09; has not been updated as of 2024-02-26):
- https://agage2.eas.gatech.edu/data_archive/global_mean/global_mean_ms.txt
- https://agage2.eas.gatech.edu/data_archive/global_mean/global_mean_md.txt

CO2, CH4 and N2O updated using Brad Hall's extended IPCC methodology. **In this pre-release we include Xin's data (personal communication)**

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.optimize import curve_fit

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
df_co2 = pd.read_csv(
    '../data/ghg_concentrations/noaa_gml/co2_annmean_gl.txt', 
    sep=r'\s+',
    comment='#', 
    names=['year', 'mean', 'unc'],
    index_col=0
)

In [4]:
df_ch4_noaa = pd.read_csv(
    '../data/ghg_concentrations/noaa_gml/ch4_annmean_gl.txt', 
    sep=r'\s+',
    comment='#', 
    names=['year', 'mean', 'unc'],
    index_col=0
)

In [5]:
df_n2o_noaa = pd.read_csv(
    '../data/ghg_concentrations/noaa_gml/n2o_annmean_gl.txt', 
    sep=r'\s+',
    comment='#', 
    names=['year', 'mean', 'unc'],
    index_col=0
)

In [6]:
df_sf6_noaa = pd.read_csv(
    '../data/ghg_concentrations/noaa_gml/sf6_annmean_gl.txt', 
    sep=r'\s+',
    comment='#', 
    names=['year', 'mean', 'unc'],
    index_col=0
)

In [7]:
df_noaa = pd.read_csv(
    '../data/ghg_concentrations/noaa_gml/noaa_2023_global_mean_mixing_ratios.csv'
)

In [8]:
df_noaa

,date,HCFC-22,CFC-113,CFC-11,HCFC-141b,CCl4,CFC-12,HCFC-142b,CH3CCl3,H-1211,H-1301,H2402,CH3Br,HFC-134a,HFC-152a,HFC-143a,HFC-125,HFC-32,HFC-365mfc,HFC-227ea
0,1992.083,98.1,79.5,265.6,ND,107.6,500.8,2.8,134.80,2.76,2.12,0.412,9.05,0.2,0.39,0.53,0.13,0.00,ND,ND
1,1992.250,100.9,79.6,266.0,ND,107.6,503.3,2.8,136.30,2.84,2.19,0.415,9.05,0.2,0.42,0.54,0.13,0.00,ND,ND
2,1992.417,101.7,80.0,266.4,ND,107.8,505.4,3.0,137.40,2.88,2.18,0.417,9.05,0.3,0.41,0.55,0.14,0.00,ND,ND
3,1992.583,102.1,80.3,266.8,ND,107.7,507.3,3.2,136.20,2.88,2.15,0.420,9.05,0.3,0.43,0.56,0.14,0.00,ND,ND
4,1992.750,103.2,80.7,267.7,ND,107.6,508.9,3.6,133.10,2.94,2.17,0.423,9.05,0.3,0.43,0.58,0.15,0.00,ND,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,2022.250,248.7,68.0,220.1,24.6,75.8,490.9,21.3,1.13,2.96,3.29,0.397,6.62,122.7,7.20,28.40,38.65,28.34,1.06,1.99
182,2022.417,248.7,67.9,219.5,24.7,75.7,490.0,21.3,1.11,2.94,3.28,0.397,6.66,123.7,7.30,28.62,39.15,29.10,1.07,2.03
183,2022.583,248.5,67.7,219.1,24.6,75.4,489.3,21.2,1.08,2.92,3.28,0.396,6.54,124.9,7.49,28.90,39.70,29.56,1.07,2.05
184,2022.750,248.8,67.6,218.9,24.6,75.1,488.7,21.1,1.07,2.91,3.28,0.397,6.39,126.2,7.46,29.49,40.69,30.45,1.07,2.08


In [9]:
df_noaa.loc[162:167,'CFC-12']

162    503.4
163    502.6
164    501.7
165    500.8
166    500.1
167    499.5
Name: CFC-12, dtype: float64

In [10]:
df_noaa[df_noaa=="ND"]=np.nan

In [11]:
df_noaa = df_noaa.rolling(6, center=True).mean()
df_noaa['YYYY'] = df_noaa.date-0.5
df_noaa.drop(df_noaa.tail(2).index,inplace=True)
df_noaa.drop(df_noaa.head(3).index,inplace=True)
df_noaa.set_index('YYYY', inplace=True)
df_noaa.drop(columns=['date'], inplace=True)
df_noaa.rename(columns={'H2402': 'H-2402'}, inplace=True)
df_noaa = df_noaa[df_noaa.index % 1 == 0]
df_noaa

,HCFC-22,CFC-113,CFC-11,HCFC-141b,CCl4,CFC-12,HCFC-142b,CH3CCl3,H-1211,H-1301,H-2402,CH3Br,HFC-134a,HFC-152a,HFC-143a,HFC-125,HFC-32,HFC-365mfc,HFC-227ea
YYYY,,,,,,,,,,,,,,,,,,,
1992.0,101.850000,80.216667,266.800000,NaN,107.633333,505.916667,3.216667,135.166667,2.888333,2.175000,0.418667,9.050000,0.266667,0.421667,0.558333,0.140000,0.000000,NaN,NaN
1993.0,105.550000,82.166667,267.933333,0.700000,107.100000,511.950000,4.383333,128.916667,3.058333,2.323333,0.434667,9.050000,0.466667,0.500000,0.626667,0.168333,0.000000,NaN,NaN
1994.0,111.400000,83.733333,267.900000,1.933333,106.066667,516.666667,5.683333,118.583333,3.295000,2.548333,0.450667,9.050000,0.900000,0.586667,0.708333,0.195000,0.000000,NaN,NaN
1995.0,116.200000,83.850000,266.183333,3.483333,105.400000,523.816667,6.600000,105.900000,3.450000,2.563333,0.462500,8.960000,1.666667,0.695000,0.845000,0.255000,0.011667,NaN,NaN
1996.0,121.483333,83.950000,265.166667,5.400000,104.283333,529.750000,7.733333,92.400000,3.613333,2.616667,0.468500,9.128333,3.050000,0.816667,1.060000,0.388333,0.050000,NaN,NaN
1997.0,126.083333,83.516667,264.050000,7.366667,103.383333,533.750000,8.700000,78.750000,3.773333,2.635000,0.474500,9.068333,5.200000,0.961667,1.345000,0.565000,0.095000,NaN,NaN
1998.0,131.400000,82.900000,262.550000,9.066667,102.466667,536.583333,9.633333,65.883333,3.886667,2.703333,0.480500,9.343333,7.650000,1.133333,1.693333,0.783333,0.153333,NaN,NaN
1999.0,136.733333,82.483333,261.183333,10.916667,101.533333,539.483333,10.683333,54.950000,4.010000,2.783333,0.486500,9.070000,10.650000,1.336667,2.071667,1.071667,0.205000,NaN,NaN
2000.0,141.533333,81.950000,259.333333,12.616667,100.216667,541.683333,11.700000,45.800000,4.111667,2.810000,0.489667,8.741667,13.933333,1.571667,2.475000,1.413333,0.258333,NaN,NaN


In [12]:
# pd.read_csv(
#     '../data/ghg_concentrations/noaa_gml/ch2cl2_GCMS_flask.txt', 
#     delim_whitespace=True,
#     skiprows=1
# )

In [13]:
# pd.read_csv(
#     '../data/ghg_concentrations/noaa_gml/OCS__GCMS_flask.txt', 
#     delim_whitespace=True,
#     skiprows=1
# )

In [14]:
# pd.read_csv(
#     '../data/ghg_concentrations/noaa_gml/pce_GCMS_flask.txt', 
#     delim_whitespace=True,
#     skiprows=1
# )

In [15]:
df_agage_ms = pd.read_csv(
    '../data/ghg_concentrations/agage/global_mean_ms.txt', 
    sep=r'\s+',
    skiprows=14,
    index_col=0
)

In [16]:
df_agage_ms = df_agage_ms.rolling(12, center=True).mean().drop([col for col in df_agage_ms.columns if '---' in col],axis=1)
df_agage_ms.drop(columns='MM', inplace=True)
df_agage_ms.set_index('YYYY', inplace=True)
df_agage_ms = df_agage_ms[df_agage_ms.index % 1 == 0]

In [17]:
df_agage_ms[df_agage_ms.index % 1 == 0]

,HFC-23,HFC-32,HFC-125,HFC-134a,HFC-143a,HFC-152a,HFC-227ea,HFC-236fa,HFC-245fa,HFC-365mfc,HFC4310mee,HCFC-22,HCFC-141b,HCFC-142b,H-1211,H-1301,H-2402,CH3Br,CH3Cl,CH2Cl2,CCl2CCl2,CHCl3,CH3CCl3,SF6,SO2F2,NF3,PFC-14,PFC-116,PFC-218,CFC-13,CFC-113,CFC-114,CFC-115
YYYY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004.0,0.000000,0.814667,3.396333,30.113750,4.899667,3.093250,0.000000,0.000000,0.000000,0.000000,0.000000,163.336750,17.451500,15.175833,4.349583,3.063000,0.483500,8.191333,534.637833,17.133167,1.836250,6.853917,21.786917,5.379500,0.218833,0.000000,0.000000,3.576167,0.405667,2.921583,79.311500,16.589667,8.343333
2005.0,0.000000,1.201000,4.048167,34.545250,5.688500,3.618583,0.000000,0.000000,0.000000,0.158667,0.000000,168.866667,17.715083,15.743667,4.370500,3.109167,0.480750,8.170333,540.391250,17.569083,1.704833,7.186000,18.107667,5.602500,1.349917,0.000000,0.000000,3.666417,0.430000,2.946083,78.617583,16.535917,8.347917
2006.0,0.000000,1.599167,4.769417,38.930000,6.613000,4.401000,0.097083,0.015667,0.122000,0.322250,0.000000,175.685917,18.103750,16.662583,4.358500,3.150500,0.474500,7.883500,533.204417,18.609750,1.668417,7.128417,15.145417,5.862000,1.424000,0.000000,50.504750,3.745667,0.459500,2.976917,77.841833,16.508167,8.364667
2007.0,5.321167,2.138500,5.613833,43.291667,7.593750,5.242833,0.430667,0.066667,0.838833,0.414000,0.000000,183.596583,18.811417,17.837667,4.333333,3.178667,0.468333,7.676750,542.339667,20.156833,1.582000,7.056917,12.596000,6.141083,1.476833,0.000000,76.419500,3.849167,0.480333,2.978417,77.131833,16.482417,8.355083
2008.0,21.894583,2.702833,6.603583,48.155083,8.637833,5.889917,0.497250,0.074000,1.031167,0.488833,0.000000,192.227500,19.465417,18.966250,4.295417,3.203417,0.463583,7.470750,545.001750,21.977083,1.592417,7.054000,10.594417,6.428250,1.516833,0.000000,77.100750,3.940083,0.501000,2.994583,76.536417,16.451000,8.368167
2009.0,22.581833,3.293667,7.663083,52.682750,9.738750,6.032000,0.573917,0.083500,1.186000,0.538500,0.000000,199.246333,19.869000,19.970000,4.246750,3.217583,0.457917,7.223667,537.817417,21.498083,1.363750,5.042167,8.892000,6.699167,1.566000,0.000000,77.687750,4.010083,0.517333,3.014583,75.873917,16.417000,8.371750
2010.0,23.220833,4.094250,8.930417,57.802000,10.883250,6.234667,0.655750,0.089250,1.331333,0.580500,0.099750,206.680750,20.489417,20.684250,4.172167,3.239250,0.451083,7.128083,541.470250,23.467667,1.364583,5.915083,7.490250,6.985833,1.632917,0.000000,78.303833,4.089500,0.537750,3.040417,75.270250,16.405417,8.382417
2011.0,24.058833,5.177917,10.435917,62.750667,12.068750,6.625417,0.735833,0.096583,1.514250,0.647500,0.213250,213.487667,21.392167,21.460083,4.099417,3.270000,0.442917,7.130083,531.228750,23.937083,1.246333,7.119417,6.230167,7.275833,1.711833,0.000000,78.990583,4.171583,0.557667,3.061917,74.622750,16.355333,8.387833
2012.0,24.943333,6.271083,12.107000,67.706750,13.415917,6.842583,0.821667,0.111917,1.710583,0.721000,0.226917,219.246250,22.471417,21.949417,4.011833,3.295500,0.436917,7.086167,538.156917,25.583083,1.177000,7.328750,5.206667,7.582000,1.798500,0.000000,79.748833,4.247500,0.571083,3.085667,74.006500,16.331000,8.404083


In [18]:
df_conc = pd.read_csv(
    '../data/ghg_concentrations/ar6_updated/ipcc_ar6_wg1.csv',
    index_col=0
)

In [19]:
df_conc.loc[2020, :] = np.nan
df_conc.loc[2021, :] = np.nan
df_conc.loc[2022, :] = np.nan
df_conc.loc[2023, :] = np.nan

In [20]:
df_co2

,mean,unc
year,,
1979,336.85,0.11
1980,338.91,0.07
1981,340.11,0.09
1982,340.86,0.03
1983,342.53,0.06
1984,344.07,0.08
1985,345.54,0.07
1986,346.97,0.07
1987,348.68,0.10


In [21]:
df_co2_ch4_n2o_update = pd.read_csv(
    '../data/ghg_concentrations/ar6_updated/co2_ch4_n2o_2019-2022.csv',
    index_col=0
)

# note CO2 is on X2007 scale in the 2022 Climate Indicators - the offset is 0.18 ppm
df_co2_ch4_n2o_update

,CO2,CH4,N2O
2019,409.9,1866.3,332.1
2020,412.3,1878.6,333.3
2021,414.5,1896.0,334.6
2022,416.9,1911.9,335.9


In [22]:
df_ch4_noaa.loc[1984:2022, 'mean']

year
1984    1644.84
1985    1657.29
1986    1670.09
1987    1682.69
1988    1693.21
1989    1704.52
1990    1714.47
1991    1724.76
1992    1735.48
1993    1736.48
1994    1742.05
1995    1748.88
1996    1751.30
1997    1754.51
1998    1765.54
1999    1772.30
2000    1773.22
2001    1771.22
2002    1772.66
2003    1777.33
2004    1777.05
2005    1774.25
2006    1774.94
2007    1781.38
2008    1787.01
2009    1793.54
2010    1798.91
2011    1803.15
2012    1808.12
2013    1813.43
2014    1822.54
2015    1834.21
2016    1843.12
2017    1849.58
2018    1857.33
2019    1866.58
2020    1878.94
2021    1895.36
2022    1911.84
Name: mean, dtype: float64

In [23]:
# difference between NOAA and NOAA/AGAGE is small
pd.concat((df_conc.loc[1984:2019, 'CH4'], df_co2_ch4_n2o_update.loc[2020:2022, 'CH4']), axis=0) - df_ch4_noaa.loc[1984:2022, 'mean']

1984   -0.1900
1985   -0.0300
1986   -0.0200
1987   -2.5615
1988   -0.2970
1989    2.1645
1990   -0.0685
1991    3.0995
1992    0.0115
1993    1.0100
1994    1.0330
1995   -0.7850
1996   -1.0640
1997   -0.8380
1998   -1.4750
1999   -0.4620
2000   -0.0770
2001    0.4320
2002   -0.1060
2003   -0.5805
2004   -0.9260
2005   -0.1080
2006   -0.5760
2007   -0.0825
2008    0.7370
2009   -0.8545
2010   -0.9955
2011    0.1760
2012    0.2520
2013    0.3345
2014    0.3830
2015   -0.2045
2016   -0.9535
2017   -0.3435
2018    0.4470
2019   -0.2525
2020   -0.3400
2021    0.6400
2022    0.0600
dtype: float64

In [24]:
df_n2o_noaa.loc[2001:2022, 'mean']

year
2001    316.36
2002    316.94
2003    317.63
2004    318.26
2005    318.91
2006    319.82
2007    320.44
2008    321.50
2009    322.27
2010    323.19
2011    324.21
2012    325.05
2013    325.94
2014    327.09
2015    328.17
2016    328.95
2017    329.74
2018    330.90
2019    331.88
2020    333.03
2021    334.31
2022    335.65
Name: mean, dtype: float64

In [25]:
# For N2O it's a bit bigger. Lindsey recommends scaling NOAA up by 1.0007
pd.concat((df_conc.loc[2001:2019, 'N2O'], df_co2_ch4_n2o_update.loc[2020:2022, 'N2O']), axis=0) - df_n2o_noaa.loc[2001:2022, 'mean']

2001    0.2635
2002    0.3315
2003    0.3525
2004    0.3775
2005    0.4385
2006    0.3705
2007    0.5015
2008    0.3485
2009    0.2840
2010    0.2460
2011    0.2360
2012    0.2680
2013    0.3085
2014    0.2795
2015    0.1325
2016    0.1740
2017    0.3000
2018    0.2905
2019    0.2110
2020    0.2700
2021    0.2900
2022    0.2500
dtype: float64

In [26]:
pd.concat((df_conc.loc[2001:2019, 'N2O'], df_co2_ch4_n2o_update.loc[2020:2022, 'N2O']), axis=0) - 1.0007 * df_n2o_noaa.loc[2001:2022, 'mean']

2001    0.042048
2002    0.109642
2003    0.130159
2004    0.154718
2005    0.215263
2006    0.146626
2007    0.277192
2008    0.123450
2009    0.058411
2010    0.019767
2011    0.009053
2012    0.040465
2013    0.080342
2014    0.050537
2015   -0.097219
2016   -0.056265
2017    0.069182
2018    0.058870
2019   -0.021316
2020    0.036879
2021    0.055983
2022    0.015045
dtype: float64

In [27]:
df_sf6_noaa

,mean,unc
year,,
1998,4.21,0.00
1999,4.38,0.00
2000,4.58,0.00
2001,4.78,0.00
2002,4.98,0.00
2003,5.23,0.00
2004,5.45,0.00
2005,5.67,0.00
2006,5.91,0.00


In [28]:
# I do think that CH4 and N2O should include AGAGE

# CO2: this is on X2019 scale. Meinshausen et al (2017) is the source of pre-1980 data in AR6. 
# Should Meinshausen have +0.18 ppm added? not clear
# anyway, let's for now just new NOAA data without adjustment
df_conc.loc[1979:2022, 'CO2'] = df_co2.loc[1979:2022, 'mean']

# For methane and N2O, the calibration scales have not changed, and we use multiple datasets, so continue with 2022 Indicators estimate and
# adjust 2023 NOAA-only value for the average of the differences between 2022 Indicators and NOAA
df_conc.loc[2020:2022, 'CH4':'N2O'] = df_co2_ch4_n2o_update.loc[2020:2022, 'CH4':'N2O']

In [29]:
# Lindsay's data, with N2O scaled
df_conc.loc[2023, 'CO2'] = 419.23
df_conc.loc[2023, 'CH4'] = 1923.3
df_conc.loc[2023, 'N2O'] = 336.74*1.0007
df_conc.loc[2023, 'SF6'] = 11.40

In [30]:
df_conc

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,HFC-236fa,HFC-245fa,HFC-365mfc,HFC-43-10mee,NF3,SF6,SO2F2,CF4,C2F6,C3F8,c-C4F8,CFC-12,CFC-11,CFC-113,CFC-114,CFC-115,CFC-13,HCFC-22,HCFC-141b,HCFC-142b,CH3CCl3,CCl4,CH3Cl,CH3Br,CH2Cl2,CHCl3,Halon-1211,Halon-1301,Halon-2402,n-C4F10,n-C5F12,n-C6F14,i-C6F14,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124
YYYY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1750,278.300000,729.200000,270.100000,0.000,0.000,0.000218,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.0000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.910000,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1850,285.500000,807.600000,272.100000,0.000,0.000,0.000218,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.0000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,285.621912,807.768974,272.181132,0.000,0.000,0.000221,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.0000,0.000021,34.050,0.000010,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1852,285.759838,808.522848,272.262214,0.000,0.000,0.000223,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.0000,0.000021,34.050,0.000021,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853,285.883757,809.768605,272.365269,0.000,0.000,0.000226,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.0000,0.000022,34.050,0.000031,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,410.070000,1866.327500,332.091000,107.591,32.414,19.980503,29.403,24.012,7.1345,1.594721,0.192,3.06,1.089484,0.288,2.053,9.9505,2.498000,85.484,4.845000,0.682,1.75,503.06,226.2015,69.799,16.013347,8.673,3.275,246.762,24.363733,22.253029,1.621,77.872,550.654105,6.491144,40.615135,8.785,3.278000,3.322,0.396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43152,NaN,NaN
2020,412.440000,1878.600000,333.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,414.700000,1896.000000,334.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_agage_md = pd.read_csv(
    '../data/ghg_concentrations/agage/global_mean_md.txt', 
    sep=r'\s+',
    skiprows=14,
    index_col=0
)

In [32]:
df_agage_md = df_agage_md.rolling(12, min_periods=12, center=True, step=12).mean().drop([col for col in df_agage_md.columns if '---' in col],axis=1)
df_agage_md.drop(columns='MM', inplace=True)
df_agage_md.set_index('YYYY', inplace=True)
df_agage_md.drop(index=np.nan, inplace=True)

In [33]:
df_agage_md

,CFC-11,CFC-12,CFC-113,CH3CCl3,CCl4,N2O,CH4,CHCl3
YYYY,,,,,,,,
1979.0,156.145417,289.108917,0.000000,76.908250,90.826000,300.794500,0.000000,0.000000
1980.0,165.689250,306.952667,0.000000,84.741250,92.631083,301.083333,0.000000,0.000000
1981.0,174.205583,321.762583,0.000000,89.771250,93.942333,301.365083,0.000000,0.000000
1982.0,182.177167,338.954167,0.000000,94.541667,94.889333,303.444250,0.000000,0.000000
1983.0,190.881667,355.958833,0.000000,98.331500,96.141333,303.635333,0.000000,0.000000
1984.0,199.263500,372.582167,23.772417,103.059750,97.334417,304.019083,0.000000,0.000000
1985.0,208.019167,390.789417,36.329333,107.198333,98.520750,304.443500,0.000000,0.000000
1986.0,219.007083,410.324500,44.588167,110.193250,100.234500,305.334917,1108.362667,0.000000
1987.0,230.677500,428.582083,50.747000,113.603250,101.314667,305.802250,1679.089917,0.000000


In [34]:
sf6_mean_offset = -(df_sf6_noaa.loc[2019, 'mean'] - df_agage_ms.loc[2019, 'SF6']).mean()
df_conc.loc[2004:2019, 'SF6'] = 0.5 * (df_sf6_noaa.loc[2004:2019, 'mean'] + df_agage_ms.loc[2004:2019, 'SF6'])
df_conc.loc[2020:2022, 'SF6'] = df_sf6_noaa.loc[2020:2022, 'mean'] + sf6_mean_offset

In [35]:
df_conc

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,HFC-236fa,HFC-245fa,HFC-365mfc,HFC-43-10mee,NF3,SF6,SO2F2,CF4,C2F6,C3F8,c-C4F8,CFC-12,CFC-11,CFC-113,CFC-114,CFC-115,CFC-13,HCFC-22,HCFC-141b,HCFC-142b,CH3CCl3,CCl4,CH3Cl,CH3Br,CH2Cl2,CHCl3,Halon-1211,Halon-1301,Halon-2402,n-C4F10,n-C5F12,n-C6F14,i-C6F14,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124
YYYY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1750,278.300000,729.200000,270.100000,0.000,0.000,0.000218,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.910000,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1850,285.500000,807.600000,272.100000,0.000,0.000,0.000218,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,285.621912,807.768974,272.181132,0.000,0.000,0.000221,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.000,0.000021,34.050,0.000010,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1852,285.759838,808.522848,272.262214,0.000,0.000,0.000223,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.000,0.000021,34.050,0.000021,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853,285.883757,809.768605,272.365269,0.000,0.000,0.000226,0.000,0.000,0.0000,0.000004,0.000,0.00,0.000000,0.000,0.000,0.000,0.000022,34.050,0.000031,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000,0.000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,410.070000,1866.327500,332.091000,107.591,32.414,19.980503,29.403,24.012,7.1345,1.594721,0.192,3.06,1.089484,0.288,2.053,9.944,2.498000,85.484,4.845000,0.682,1.75,503.06,226.2015,69.799,16.013347,8.673,3.275,246.762,24.363733,22.253029,1.621,77.872,550.654105,6.491144,40.615135,8.785,3.278000,3.322,0.396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43152,NaN,NaN
2020,412.440000,1878.600000,333.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,414.700000,1896.000000,334.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
species = [
    'HFC-134a', 'HFC-32', 'HFC-125', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-365mfc', 'HCFC-22', 
    'HCFC-141b', 'HCFC-142b', 'CH3CCl3', 'CH3Br', 'H-1211', 'H-1301', 'H-2402'
]
names = {specie: specie for specie in species}
first = {specie: 2004 for specie in species}
names['H-1211'] = 'Halon-1211'
names['H-1301'] = 'Halon-1301'
names['H-2402'] = 'Halon-2402'
first['HFC-365mfc'] = 2008
first['HFC-227ea'] = 2008

In [37]:
for specie in species:
    mean_offset = -(df_noaa.loc[2020, specie] - df_agage_ms.loc[2020, specie]).mean()
    df_conc.loc[first[specie]:2020, names[specie]] = 0.5 * (df_noaa.loc[first[specie]:2020, specie] + df_agage_ms.loc[first[specie]:2020, specie])
    df_conc.loc[2021, names[specie]] = df_noaa.loc[2021, specie] + mean_offset

In [38]:
df_conc

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,HFC-236fa,HFC-245fa,HFC-365mfc,HFC-43-10mee,NF3,SF6,SO2F2,CF4,C2F6,C3F8,c-C4F8,CFC-12,CFC-11,CFC-113,CFC-114,CFC-115,CFC-13,HCFC-22,HCFC-141b,HCFC-142b,CH3CCl3,CCl4,CH3Cl,CH3Br,CH2Cl2,CHCl3,Halon-1211,Halon-1301,Halon-2402,n-C4F10,n-C5F12,n-C6F14,i-C6F14,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124
YYYY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1750,278.300000,729.200000,270.100000,0.000000,0.000,0.000218,0.000000,0.000000,0.000000,0.000004,0.000,0.00,0.00000,0.000,0.000,0.000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.025,457.000000,5.299998,6.910000,4.800,0.004447,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1850,285.500000,807.600000,272.100000,0.000000,0.000,0.000218,0.000000,0.000000,0.000000,0.000004,0.000,0.00,0.00000,0.000,0.000,0.000,0.000021,34.050,0.000001,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000000,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851,285.621912,807.768974,272.181132,0.000000,0.000,0.000221,0.000000,0.000000,0.000000,0.000004,0.000,0.00,0.00000,0.000,0.000,0.000,0.000021,34.050,0.000010,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000000,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1852,285.759838,808.522848,272.262214,0.000000,0.000,0.000223,0.000000,0.000000,0.000000,0.000004,0.000,0.00,0.00000,0.000,0.000,0.000,0.000021,34.050,0.000021,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000000,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853,285.883757,809.768605,272.365269,0.000000,0.000,0.000226,0.000000,0.000000,0.000000,0.000004,0.000,0.00,0.00000,0.000,0.000,0.000,0.000022,34.050,0.000031,0.000,0.00,0.00,0.0000,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.025,457.000000,5.299998,6.912761,4.800,0.004447,0.000000,0.000000,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,410.070000,1866.327500,332.091000,107.606417,32.414,20.581125,29.596417,24.004208,7.157250,1.598292,0.192,3.06,1.08225,0.288,2.053,9.944,2.498000,85.484,4.845000,0.682,1.75,503.06,226.2015,69.799,16.013347,8.673,3.275,246.760833,24.381208,22.220250,1.629000,77.872,550.654105,6.491208,40.615135,8.785,3.260250,3.318542,0.398625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43152,NaN,NaN
2020,412.440000,1878.600000,333.300000,112.988500,NaN,23.658333,32.783417,25.557917,7.092792,1.750542,NaN,NaN,1.09800,NaN,NaN,10.278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.894417,24.522667,21.954792,1.409083,NaN,NaN,6.598458,NaN,NaN,3.158042,3.317208,0.393542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,414.700000,1896.000000,334.600000,118.727000,NaN,28.450000,36.728500,27.502500,7.392250,1.966083,NaN,NaN,1.20100,NaN,NaN,10.638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249.072167,24.628667,22.076250,1.254833,NaN,NaN,6.461917,NaN,NaN,3.119417,3.374417,0.385750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
species = [
    'CFC-11', 'CFC-12', 'CCl4'
]
names = {specie: specie for specie in species}
first = {specie: 1992 for specie in species}
last = {specie: 2019 for specie in species}

In [40]:
df_conc.loc[2020:2022, 'CH3CCl3'] = df_noaa.loc[2020:2022, 'CH3CCl3']

In [41]:
for specie in species:
    mean_offset = -(df_noaa.loc[last[specie], specie] - df_agage_md.loc[last[specie], specie]).mean()
    df_conc.loc[first[specie]:last[specie], names[specie]] = 0.5 * (df_noaa.loc[first[specie]:last[specie], specie] + df_agage_md.loc[first[specie]:last[specie], specie])
    df_conc.loc[last[specie]+1:2022, names[specie]] = df_noaa.loc[last[specie]+1:, specie] + mean_offset

In [42]:
df_conc.loc[2007:2020, 'CF4'] = df_agage_ms.loc[2007:2020, 'PFC-14']

In [43]:
df_conc.loc[2004:2020, 'C2F6'] = df_agage_ms.loc[2004:2020, 'PFC-116']
df_conc.loc[2004:2020, 'C3F8'] = df_agage_ms.loc[2004:2020, 'PFC-218']

In [44]:
df_conc.loc[2008:2020, 'HFC-23'] = df_agage_ms.loc[2008:2020, 'HFC-23']

In [45]:
df_conc.loc[2007:2020, 'HFC-236fa'] = df_agage_ms.loc[2007:2020, 'HFC-236fa']

In [46]:
df_conc.loc[2007:2020, 'HFC-245fa'] = df_agage_ms.loc[2007:2020, 'HFC-245fa']

In [47]:
df_conc.loc[2008:2020, 'HFC-23'] = df_agage_ms.loc[2008:2020, 'HFC-23']

In [48]:
df_agage_ms.loc[2008:2020, 'HFC4310mee']

YYYY
2008.0    0.000000
2009.0    0.000000
2010.0    0.099750
2011.0    0.213250
2012.0    0.226917
2013.0    0.239250
2014.0    0.245500
2015.0    0.254000
2016.0    0.264750
2017.0    0.272417
2018.0    0.278833
2019.0    0.288917
2020.0    0.298417
Name: HFC4310mee, dtype: float64

In [49]:
df_conc.loc[2011:2020, 'HFC-43-10mee'] = df_agage_ms.loc[2011:2020, 'HFC4310mee']

In [50]:
df_agage_ms.loc[:, 'SO2F2']

YYYY
2004.0    0.218833
2005.0    1.349917
2006.0    1.424000
2007.0    1.476833
2008.0    1.516833
2009.0    1.566000
2010.0    1.632917
2011.0    1.711833
2012.0    1.798500
2013.0    1.900333
2014.0    2.019000
2015.0    2.106667
2016.0    2.196833
2017.0    2.322417
2018.0    2.407500
2019.0    2.500167
2020.0    2.581000
Name: SO2F2, dtype: float64

In [51]:
df_conc.loc[2004:2020, 'CFC-13'] = df_agage_ms.loc[2004:2020, 'CFC-13']
df_conc.loc[2004:2020, 'CFC-114'] = df_agage_ms.loc[2004:2020, 'CFC-114']
df_conc.loc[2004:2020, 'CFC-115'] = df_agage_ms.loc[2004:2020, 'CFC-115']

In [52]:
df_conc.loc[2016:2020, 'NF3'] = df_agage_ms.loc[2016:2020, 'NF3']

In [53]:
df_conc.loc[2005:2020, 'SO2F2'] = df_agage_ms.loc[2005:2020, 'SO2F2']

In [54]:
df_conc.loc[2004:2020, 'CH3Cl'] = df_agage_ms.loc[2004:2020, 'CH3Cl']

In [55]:
df_conc.loc[2004:2020, 'CH2Cl2'] = df_agage_ms.loc[2004:2020, 'CH2Cl2']

In [56]:
df_conc.loc[2004:2020, 'CHCl3'] = df_agage_ms.loc[2004:2020, 'CHCl3']

In [57]:
df_conc.loc[1850:1989, 'i-C6F14'] = 0
df_conc.loc[1990:2015, 'i-C6F14'].interpolate(inplace=True)

/tmp/ipykernel_7721/3131240140.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_conc.loc[1990:2015, 'i-C6F14'].interpolate(inplace=True)


In [58]:
df_conc.loc[1850:1977, 'CFC-112'] = 0
df_conc.loc[1850:1977, 'CFC-112a'] = 0
df_conc.loc[1850:1977, 'CFC-113a'] = 0
df_conc.loc[1850:1977, 'CFC-114a'] = 0
df_conc.loc[1850:1979, 'HCFC-133a'] = 0
df_conc.loc[1850:1999, 'HCFC-31'] = 0
df_conc.loc[1850:2003, 'HCFC-124'] = 0

In [59]:
# Function to curve fit to the data
def linear(x, c, d):
    return c * x + d

# Initial parameter guess, just to kick off the optimization
guess = (1, 0)

# Place to store function parameters for each column
col_params = {}

# Curve fit each column
for col in df_conc.columns:
    # Create copy of data to remove NaNs for curve fitting
    fit_df = df_conc[col].dropna()

    # Get x & y
    x = fit_df.index.astype(float).values[-5:]
    y = fit_df.values[-5:]
    print (col, x, y)
    # Curve fit column and get curve parameters
    params = curve_fit(linear, x, y, guess)
    # Store optimized parameters
    col_params[col] = params[0]

# Extrapolate each column
for col in df_conc.columns:
    # Get the index values for NaNs in the column
    x = df_conc[pd.isnull(df_conc[col])].index.astype(float).values
    print(col, x)
    # Extrapolate those points with the fitted function
    df_conc[col][x] = linear(x, *col_params[col])

CO2 [2019. 2020. 2021. 2022. 2023.] [410.07 412.44 414.7  417.07 419.23]
CH4 [2019. 2020. 2021. 2022. 2023.] [1866.3275 1878.6    1896.     1911.9    1923.3   ]
N2O [2019. 2020. 2021. 2022. 2023.] [332.091    333.3      334.6      335.9      336.975718]
HFC-134a [2017. 2018. 2019. 2020. 2021.] [ 95.722      101.87       107.60641667 112.9885     118.727     ]
HFC-23 [2016. 2017. 2018. 2019. 2020.] [28.921      29.92716667 31.17158333 32.4755     33.72883333]
HFC-32 [2017. 2018. 2019. 2020. 2021.] [14.522625   17.42370833 20.581125   23.65833333 28.45      ]
HFC-125 [2017. 2018. 2019. 2020. 2021.] [23.32570833 26.441875   29.59641667 32.78341667 36.7285    ]
HFC-143a [2017. 2018. 2019. 2020. 2021.] [20.84254167 22.42775    24.00420833 25.55791667 27.5025    ]
HFC-152a [2017. 2018. 2019. 2020. 2021.] [6.80058333 6.98879167 7.15725    7.09279167 7.39225   ]
HFC-227ea [2017. 2018. 2019. 2020. 2021.] [1.32754167 1.45970833 1.59829167 1.75054167 1.96608333]
HFC-236fa [2016. 2017. 2018. 2019.

/tmp/ipykernel_7721/537674771.py:31: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_conc[col][x] = linear(x, *col_params[col])
/tmp/ipykernel_7721/537674771.py:31: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!

In [60]:
df_conc

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,HFC-236fa,HFC-245fa,HFC-365mfc,HFC-43-10mee,NF3,SF6,SO2F2,CF4,C2F6,C3F8,c-C4F8,CFC-12,CFC-11,CFC-113,CFC-114,CFC-115,CFC-13,HCFC-22,HCFC-141b,HCFC-142b,CH3CCl3,CCl4,CH3Cl,CH3Br,CH2Cl2,CHCl3,Halon-1211,Halon-1301,Halon-2402,n-C4F10,n-C5F12,n-C6F14,i-C6F14,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124
YYYY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1750,278.300000,729.200000,270.100000,0.000000,0.000000,0.000218,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000021,34.050000,0.000001,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,457.000000,5.299998,6.910000,4.800000,0.004447,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.00000,0.000,0.00000,0.0000,0.000,0.0000,0.000000,0.0000,0.000
1850,285.500000,807.600000,272.100000,0.000000,0.000000,0.000218,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000021,34.050000,0.000001,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,457.000000,5.299998,6.912761,4.800000,0.004447,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.00000,0.000,0.00000,0.0000,0.000,0.0000,0.000000,0.0000,0.000
1851,285.621912,807.768974,272.181132,0.000000,0.000000,0.000221,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000021,34.050000,0.000010,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,457.000000,5.299998,6.912761,4.800000,0.004447,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.00000,0.000,0.00000,0.0000,0.000,0.0000,0.000000,0.0000,0.000
1852,285.759838,808.522848,272.262214,0.000000,0.000000,0.000223,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000021,34.050000,0.000021,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,457.000000,5.299998,6.912761,4.800000,0.004447,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.00000,0.000,0.00000,0.0000,0.000,0.0000,0.000000,0.0000,0.000
1853,285.883757,809.768605,272.365269,0.000000,0.000000,0.000226,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000022,34.050000,0.000031,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,457.000000,5.299998,6.912761,4.800000,0.004447,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.00000,0.000,0.00000,0.0000,0.000,0.0000,0.000000,0.0000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,410.070000,1866.327500,332.091000,107.606417,32.475500,20.581125,29.596417,24.004208,7.157250,1.598292,0.196083,3.096750,1.082250,0.288917,2.077250,9.944,2.500167,85.558667,4.847667,0.684583,1.7500,502.955167,226.220500,69.7990,16.295083,8.677583,3.279083,246.760833,24.381208,22.220250,1.629000,78.079625,540.818417,6.491208,36.772250,8.781500,3.260250,3.318542,0.398625,0.2032,0.1492,0.2296,0.0676,0.11836,0.096,0.40910,0.0696,0.873,1.0289,0.431520,0.0884,0.954
2020,412.440000,1878.600000,333.300000,112.988500,33.728833,23.658333,32.783417,25.557917,7.092792,1.750542,0.207167,3.278500,1.098000,0.298417,2.315250,10.278,2.581000,86.482750,4.937250,0.704083,1.8114,500.410333,223.441000,69.2613,16.276417,8.706917,3.316917,247.894417,24.522667,21.954792,1.400000,76.125917,546.026000,6.598458,37.785000,8.715333,3.158042,3.317208,0.393542,0.2073,0.1528,0.2312,0.0690,0.11984,0.097,0.40725,0.0704,0.920,1.0262,0.450585,0.0884,0.938
2021,414.700000,1896.000000,334.600000,118.727000,34.894017,28.450000,36.728500,27.502500,7.392250,1.966083,0.223492,3.508475,1.201000,0.3058

In [61]:
os.makedirs('../output', exist_ok = True)
df_conc.to_csv('../output/ghg_concentrations_1750-2023.csv')

## Aggregated categories

In [62]:
gases_hfcs = [
    'HFC-134a',
    'HFC-23', 
    'HFC-32', 
    'HFC-125',
    'HFC-143a', 
    'HFC-152a', 
    'HFC-227ea', 
    'HFC-236fa', 
    'HFC-245fa', 
    'HFC-365mfc',
    'HFC-43-10mee',
]
gases_montreal = [
    'CFC-12',
    'CFC-11',
    'CFC-113',
    'CFC-114',
    'CFC-115',
    'CFC-13',
    'HCFC-22',
    'HCFC-141b',
    'HCFC-142b',
    'CH3CCl3',
    'CCl4',  # yes
    'CH3Cl',  # no
    'CH3Br',  # yes
    'CH2Cl2',  # no!
    'CHCl3',  # no
    'Halon-1211',
    'Halon-1301',
    'Halon-2402',
    'CFC-112',
    'CFC-112a',
    'CFC-113a',
    'CFC-114a',
    'HCFC-133a',
    'HCFC-31',
    'HCFC-124'
]
gases_pfc = [
    'CF4',
    'C2F6',
    'C3F8',
    'c-C4F8',
    'n-C4F10',
    'n-C5F12',
    'n-C6F14',
    'i-C6F14',
    'C7F16',
    'C8F18',
]

In [63]:
# source: Hodnebrog et al 2020 https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019RG000691
radeff = {
    'HFC-125':      0.23378,
    'HFC-134a':     0.16714,
    'HFC-143a':     0.168,
    'HFC-152a':     0.10174,
    'HFC-227ea':    0.27325,
    'HFC-23':       0.19111,
    'HFC-236fa':    0.25069,
    'HFC-245fa':    0.24498,
    'HFC-32':       0.11144,
    'HFC-365mfc':   0.22813,
    'HFC-43-10mee': 0.35731,
    'NF3':          0.20448,
    'C2F6':         0.26105,
    'C3F8':         0.26999,
    'n-C4F10':      0.36874,
    'n-C5F12':      0.4076,
    'n-C6F14':      0.44888,
    'i-C6F14':      0.44888,
    'C7F16':        0.50312,
    'C8F18':        0.55787,
    'CF4':          0.09859,
    'c-C4F8':       0.31392,
    'SF6':          0.56657,
    'SO2F2':        0.21074,
    'CCl4':         0.16616,
    'CFC-11':       0.25941,
    'CFC-112':      0.28192,
    'CFC-112a':     0.24564,
    'CFC-113':      0.30142,
    'CFC-113a':     0.24094, 
    'CFC-114':      0.31433,
    'CFC-114a':     0.29747,
    'CFC-115':      0.24625,
    'CFC-12':       0.31998,
    'CFC-13':       0.27752,
    'CH2Cl2':       0.02882,
    'CH3Br':        0.00432,
    'CH3CCl3':      0.06454,
    'CH3Cl':        0.00466,
    'CHCl3':        0.07357,
    'HCFC-124':     0.20721,
    'HCFC-133a':    0.14995,
    'HCFC-141b':    0.16065,
    'HCFC-142b':    0.19329,
    'HCFC-22':      0.21385,
    'HCFC-31':      0.068,
    'Halon-1202':   0,       # not in dataset
    'Halon-1211':   0.30014,
    'Halon-1301':   0.29943,
    'Halon-2402':   0.31169,
    'CO2':          0,       # different relationship
    'CH4':          0,       # different relationship
    'N2O':          0        # different relationship
}

In [64]:
pfc_hfc134a_eq_1750 = 0
for gas in gases_pfc:
    pfc_hfc134a_eq_1750 = pfc_hfc134a_eq_1750 + (df_conc.loc[1750, gas] * radeff[gas] / radeff['CF4'])
hfc_hfc134a_eq_1750 = 0
for gas in gases_hfcs:
    hfc_hfc134a_eq_1750 = hfc_hfc134a_eq_1750 + (df_conc.loc[1750, gas] * radeff[gas] / radeff['HFC-134a'])
montreal_cfc12_eq_1750 = 0
for gas in gases_montreal:
    montreal_cfc12_eq_1750 = montreal_cfc12_eq_1750 + (df_conc.loc[1750, gas] * radeff[gas] / radeff['CFC-12'])

In [65]:
pfc_hfc134a_eq_1750, hfc_hfc134a_eq_1750, montreal_cfc12_eq_1750

(34.05000340932363, 0.00015207245048766408, 8.47017581128727)

In [66]:
pfc_hfc134a_eq_1850 = 0
for gas in gases_pfc:
    pfc_hfc134a_eq_1850 = pfc_hfc134a_eq_1850 + (df_conc.loc[1850, gas] * radeff[gas] / radeff['CF4'])
hfc_hfc134a_eq_1850 = 0
for gas in gases_hfcs:
    hfc_hfc134a_eq_1850 = hfc_hfc134a_eq_1850 + (df_conc.loc[1850, gas] * radeff[gas] / radeff['HFC-134a'])
montreal_cfc12_eq_1850 = 0
for gas in gases_montreal:
    montreal_cfc12_eq_1850 = montreal_cfc12_eq_1850 + (df_conc.loc[1850, gas] * radeff[gas] / radeff['CFC-12'])

In [67]:
pfc_hfc134a_eq_1850, hfc_hfc134a_eq_1850, montreal_cfc12_eq_1850

(34.05000340932363, 0.00015207245048766408, 8.470424514631054)

In [68]:
pfc_hfc134a_eq_2019 = 0
for gas in gases_pfc:
    pfc_hfc134a_eq_2019 = pfc_hfc134a_eq_2019 + (df_conc.loc[2019, gas] * radeff[gas] / radeff['CF4'])
hfc_hfc134a_eq_2019 = 0
for gas in gases_hfcs:
    hfc_hfc134a_eq_2019 = hfc_hfc134a_eq_2019 + (df_conc.loc[2019, gas] * radeff[gas] / radeff['HFC-134a'])
montreal_cfc12_eq_2019 = 0
for gas in gases_montreal:
    montreal_cfc12_eq_2019 = montreal_cfc12_eq_2019 + (df_conc.loc[2019, gas] * radeff[gas] / radeff['CFC-12'])

In [69]:
pfc_hfc134a_eq_2019, hfc_hfc134a_eq_2019, montreal_cfc12_eq_2019

(109.71860032003106, 237.8837107240437, 1031.8000571243554)

In [70]:
pfc_hfc134a_eq_2022 = 0
for gas in gases_pfc:
    pfc_hfc134a_eq_2022 = pfc_hfc134a_eq_2022 + (df_conc.loc[2022, gas] * radeff[gas] / radeff['CF4'])
hfc_hfc134a_eq_2022 = 0
for gas in gases_hfcs:
    hfc_hfc134a_eq_2022 = hfc_hfc134a_eq_2022 + (df_conc.loc[2022, gas] * radeff[gas] / radeff['HFC-134a'])
montreal_cfc12_eq_2022 = 0
for gas in gases_montreal:
    montreal_cfc12_eq_2022 = montreal_cfc12_eq_2022 + (df_conc.loc[2022, gas] * radeff[gas] / radeff['CFC-12'])

In [71]:
pfc_hfc134a_eq_2022, hfc_hfc134a_eq_2022, montreal_cfc12_eq_2022

(114.19443264659608, 287.4163706218587, 1015.7670940994637)